In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/MA')
print(os.listdir(os.getcwd()))
import json
import pandas as pd
import gzip
import numpy as np
import sys
import matplotlib.pyplot as plt
import time
from __future__ import absolute_import, division, print_function
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.utils import Sequence
from sklearn.metrics import log_loss, roc_auc_score, confusion_matrix

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
['SIG Proceedings Template-Jan2015 Zip', '.DS_Store', 'we_data.zip', 'sig-proceedings-template-may2015-zip.zip', 'data', 'click_predictions', 'bid_predictions', 'payprice_predictions', 'Click Prediction Grid Search.gsheet', 'Payprice_Prediction.ipynb', 'Click_Prediction.ipynb', 'data2', 'League_of_Agents.ipynb', 'MA_Feature_Engineering.ipynb', 'Data_Preprocessing.ipynb']


In [0]:
folder='data/'
for file in ['validation','test','train']:
  data=pd.read_csv(folder+file+'.csv')
  print(file+' is successfully loaded')
  print('No. of rows: ',len(data))

  data=data.drop(columns=['bidid','userid','IP','urlid','keypage'])
  print('columns "bidid", "userid", "IP", "urlid" and "keypage" are dropped')
  try:
    data=data.drop(columns=['bidprice'])
    print('column "bidprice" is dropped')
  except:
    pass

  for i in range(len(data)):
    if not isinstance(data.at[i,'domain'],str):
      if isinstance(data.at[i,'url'],str):
        data.at[i,'domain']=data.at[i,'url']
      else:
        data.at[i,'domain']='Na'
    if i%200000==0:
          print(int(i/100000),'/',int(len(data)/100000))
  print('column "url" is pasted into column "domain" where "domain" is nan')
  print('nan is reformated as string "Na" in column "domain"')
  data=data.drop(columns='url')
  print('column "url" is dropped')

  for i in range(len(data)):
    if np.isnan(data.at[i,'adexchange']):
      data.at[i,'adexchange']=-1
    if i%200000==0:
      print(int(i/100000),'/',int(len(data)/100000))
  data['adexchange']=data['adexchange'].astype(int)
  print('column "adexchange" is reformated from float to int')

  data['useros']=''
  data['userbrowser']=''
  for i in range(len(data)):
    user_agent=data.at[i,'useragent'].split('_')
    data.at[i,'useros']=user_agent[0]
    data.at[i,'userbrowser']=user_agent[1]
    if i%200000==0:
      print(int(i/100000),'/',int(len(data)/100000))
  print('column "useragent" is divided into columns "useros" and "userbrowser"')
  data=data.drop(columns='useragent')
  print('column "useragent" is dropped')

  data['slotsize']=''
  for i in range(len(data)):
    data.at[i,'slotsize']=str(data.at[i,'slotwidth'])+'*'+str(data.at[i,'slotheight'])
    if i%200000==0:
      print(int(i/100000),'/',int(len(data)/100000))
  print('columns "slotwidth" and "slotheight" are combined into column "slotsize"')
  data=data.drop(columns=['slotwidth','slotheight'])
  print('columns "slotwidth" and "slotheight" are dropped')

  data['slotid0']='Na'
  data['slotid1']='Na'
  data['slotid2']='Na'
  for i in range(len(data)):
    slotid=str(data.at[i,'slotid'])
    if slotid[0]=='_':
      slotid=slotid[1:]
    if '_' in slotid:
      slotid=slotid.split('_',1)
      data.at[i,'slotid0']=slotid[0]
      data.at[i,'slotid1']=slotid[1]
    elif len(slotid)==1:
      data.at[i,'slotid0']=slotid
    else:
      data.at[i,'slotid2']=slotid
    if i%200000==0:
      print(int(i/100000),'/',int(len(data)/100000))
  print('column "slotid" is divided into columns "slotid0", "slotid1" and "slotid2"')
  data=data.drop(columns='slotid')
  print('column "slotid" is dropped')


  usertag_set=sorted(list(set(','.join([n for n in data.usertag if isinstance(n,str)]).split(','))))
  for tag in usertag_set:
    data['usertag_'+tag]=False
  for i in range(len(data)):
    if isinstance(data.at[i,'usertag'],str):
      for tag in data.at[i,'usertag'].split(','):
        data.at[i,'usertag_'+tag]=True
    if i%200000==0:
      print(int(i/100000),'/',int(len(data)/100000))
  print('column "usertag" is divided into ',len(usertag_set),' binary columns')
  data=data.drop(columns='usertag')
  print('column "usertag" is dropped')
  
  if file=='validation':
    for i in range(len(data)):
      if data.at[i,'slotprice']==288:
        data.at[i,'slotprice']=287

  with open(folder+file+'1.csv','w') as f:
    f.write(data.to_csv(index=False))
    print('new copy of '+file+' is saved')

validation is successfully loaded
No. of rows:  303925
columns "bidid", "userid", "IP", "urlid" and "keypage" are dropped
column "bidprice" is dropped
0 / 3
2 / 3
column "url" is pasted into column "domain" where "domain" is nan
nan is reformated as string "Na" in column "domain"
column "url" is dropped
0 / 3
2 / 3
column "adexchange" is reformated from float to int
0 / 3
2 / 3
column "useragent" is divided into columns "useros" and "userbrowser"
column "useragent" is dropped
0 / 3
2 / 3
columns "slotwidth" and "slotheight" are combined into column "slotsize"
columns "slotwidth" and "slotheight" are dropped
0 / 3
2 / 3
column "slotid" is divided into columns "slotid0", "slotid1" and "slotid2"
column "slotid" is dropped
0 / 3
2 / 3
column "usertag" is divided into  68  binary columns
column "usertag" is dropped
new copy of validation is saved
test is successfully loaded
No. of rows:  303375
columns "bidid", "userid", "IP", "urlid" and "keypage" are dropped
0 / 3
2 / 3
column "url" is pa

In [0]:
folder='data/'
train1=pd.read_csv(folder+'train1'+'.csv',dtype={'slotvisibility':str,'slotformat':str,'creative':str,'slotid2':str})
print('train1'+' is successfully loaded','No. of rows: ',len(train1))
#########################
for ds_rate in [0.01]:
#########################
  folder='data/'
  data=train1
  drop_list=[]
  for i in range(len(data)):
    if data.at[i,'click']==1:
      drop_list.append(i)
  positive_data=data.iloc[drop_list]
  negative_data=data.drop(drop_list)
  positive_data.index=range(len(positive_data))
  negative_data.index=range(len(negative_data))
  negative_data=negative_data.iloc[np.random.choice(len(negative_data),int(len(negative_data)*ds_rate),replace=False)]
  negative_data.index=range(len(negative_data))
  data=positive_data.append(negative_data)
  data.index=range(len(data))
  data=data.iloc[np.random.permutation(len(data))]
  data.index=range(len(data))
  print('data negative-downsampled',len(data))
  
  data=data.drop(columns=['domain','slotid1','slotid2'])

#   threshold=1
#   for column in ['domain','slotid2']:
#     print('column "'+column+'" has',len(list(set(data[column]))),'unique instances')
#     data=data.sort_values(by=column)
#     data.index=range(len(data)) 
#     i=0
#     i_start=i
#     while i<len(data):
#       i+=1
#       if i==len(data) or not data.at[i,column]==data.at[i_start,column]:
#         if i-i_start<=threshold:
#           if data.at[i_start,'click']==0:
#             for j in range(i_start,i):
#               data.at[j,column]='Na'
#         i_start=i
#     print('instances with frequency <= '+str(threshold)+' are changed to "Na" in column "'+column+'"')
#     print('column "'+column+'" has',len(list(set(data[column]))),'unique instances')

#   dic1=dict.fromkeys(sorted(list(set(data['domain']))))
#   dic2=dict.fromkeys(sorted(list(set(data['slotid2']))))
#   i=0
#   for key in dic1.keys():
#     dic1[key]=i
#     i+=1
#   j=0
#   for key in dic2.keys():
#     dic2[key]=j
#     j+=1
#   co=np.zeros([i,j]).astype(bool)
#   for k in range(len(data)):
#     co[dic1[data.at[k,'domain']],dic2[data.at[k,'slotid2']]]=1
#   duplicate_list=np.nonzero((np.outer(np.sum(co,axis=1),np.sum(co,axis=0))*co)==1)[1]
#   print(len(duplicate_list),'duplicate instances found between column "domain" and column "slotid2"')
#   drop_list=[]
#   for key in dic2.keys():
#     if dic2[key] in duplicate_list:
#       drop_list.append(key)
#   for key in drop_list:
#     del dic2[key]
#   for i in range(len(data)):
#     try:
#       dummy=dic2[data.at[i,'slotid2']]
#     except:
#       data.at[i,'slotid2']='Na'
#   print('duplicate instances in column "slotid2" are changed to "Na"')
#   print('column "slotid2" has',len(list(set(data.slotid2))),'unique instances')

  dic={}
  for column in data.columns:
    if not ('usertag' in column or 'click' in column or 'payprice' in column):
      for item in sorted(list(set(data[column]))):
        if not (item==-1 or item=='Na'):
#           if column in ['domain','slotid1','slotid2']:
#             dic[column+'_'+str(item)]=False
#           else:
            dic[column+'_'+str(item)]=True
    elif 'usertag' in column:
      dic[column]=True
  print('Total unique feature instances:',len(dic.keys()))

  datasets={}
  datasets['train']=data
  datasets['validation']=pd.read_csv(folder+'validation'+'1.csv',dtype={'slotvisibility':str,'slotformat':str,'creative':str,'slotid2':str})
  datasets['test']=pd.read_csv(folder+'test'+'1.csv',dtype={'slotvisibility':str,'slotformat':str,'creative':str,'slotid2':str})
  
  #################################
  folder=folder+'minimalfeatures/'
  #################################
  
#   for dataset in ['validation','test']:
#     data=datasets[dataset]
#     print(dataset,'No. of rows: ',len(data))
#     for column in ['domain','slotid1','slotid2']:
#       for i in range(len(data)):
#         try:
#           dummy=dic[column+'_'+str(data.at[i,column])]
#           dic[column+'_'+str(data.at[i,column])]=True
#         except:
#           data.at[i,column]='Na'    
#       print('instances which do not appear in training set are changed to "Na" in column "'+column+'"')
  
#   drop_list=[]
#   for key in dic.keys():
#     if dic[key]==False:
#       drop_list.append(key)
#   for key in drop_list:
#     del dic[key]
#   print('drop list',len(drop_list))
#   print('Total unique feature instances:',len(dic.keys()))
  i=0
  for key in dic.keys():
    dic[key]=i
    i+=1
  with open(folder+str(ds_rate)+'/'+'dic_'+str(ds_rate)+'.json', 'w') as f:
    json.dump(dic, f)

  batch_size=int(72000/len(dic.keys()))*10000
  print('batch size',batch_size)
  category_columns=[column for column in datasets['train'].columns if not ('usertag' in column or 'click' in column or 'payprice' in column)]
  binary_columns=[column for column in datasets['train'].columns if 'usertag' in column]
  n_category_columns=len(category_columns)
  n_binary_category_columns=dic[binary_columns[0]]

  for dataset in ['train','validation','test']:
    print(dataset)
    data=datasets[dataset]
    try:
      data=data.drop(columns=['domain','slotid1','slotid2'])
    except:
      pass
    
    if not dataset=='test':
      click=np.asarray(data.click).astype(bool)
      payprice=np.asarray(data.payprice).astype(int)
      data=data.drop(columns=['click','payprice'])
      click.tofile(folder+str(ds_rate)+'/'+'_'.join([dataset,str(ds_rate),'click'])+'.bin')
      payprice.tofile(folder+str(ds_rate)+'/'+'_'.join([dataset,str(ds_rate),'payprice'])+'.bin')
      print('click and payprice saved')
    data=np.asarray(data)
    for i in range(int(np.ceil(len(data)/float(batch_size)))):
      x=data[i*batch_size:min((i+1)*batch_size,len(data))]
      binary_x=np.zeros([len(x),n_binary_category_columns]).astype(bool)
      for j in range(len(x)):
        for k in range(n_category_columns):
          try:
            binary_x[j,dic[category_columns[k]+'_'+str(x[j,k])]]=True
          except:
            pass
      binary_x=np.hstack((binary_x,x[:,n_category_columns:])).astype(bool)
      binary_x.tofile(folder+str(ds_rate)+'/'+'_'.join([dataset,str(ds_rate),'x',str(i),str(len(x))])+'.bin')
      print(i,'x')

train1 is successfully loaded No. of rows:  2430981
data negative-downsampled 26084
Total unique feature instances: 886
batch size 810000
train
click and payprice saved
0 x
validation
click and payprice saved
0 x
test
0 x


In [0]:
folder='data/'
data=pd.read_csv(folder+'validation1'+'.csv',dtype={'slotvisibility':str,'slotformat':str,'creative':str,'slotid2':str})
print('No. of rows: ',len(data))
drop_list=[]
for i in range(len(data)):
  if data.at[i,'click']==1:
    drop_list.append(i)
positive_data=data.iloc[drop_list]
positive_data.index=range(len(positive_data))
with open(folder+'validation1'+'_positive'+'.csv','w') as f:
  f.write(positive_data.to_csv(index=False))
print(len(positive_data))

No. of rows:  303925
202


In [0]:
payprice_bin_width=10
slotprice_bin_width=10
folder='data/'
data=pd.read_csv(folder+'train1_positive'+'.csv',dtype={'slotvisibility':str,'slotformat':str,'creative':str,'slotid2':str})
data.slotprice=np.ceil(np.minimum(data.slotprice,210)/slotprice_bin_width).astype(int)
for column in ['domain','slotid2']:
  print('column "'+column+'" has',len(list(set(data[column]))),'unique instances')
  
data=data.drop(columns=[])
  
dic={}
for column in data.columns:
  if not ('usertag' in column or 'click' in column or 'payprice' in column):
    for item in sorted(list(set(data[column]))):
      if not (item==-1 or item=='Na'):
#         if column in ['domain','slotid1','slotid2']:
#           dic[column+'_'+str(item)]=False
#         else:
          dic[column+'_'+str(item)]=True
  elif 'usertag' in column:
    dic[column]=True
print('Total unique feature instances:',len(dic.keys()))

datasets={}
datasets['train']=data
datasets['validation']=pd.read_csv(folder+'validation'+'1.csv',dtype={'slotvisibility':str,'slotformat':str,'creative':str,'slotid2':str}).drop(columns=[])
datasets['test']=pd.read_csv(folder+'test'+'1.csv',dtype={'slotvisibility':str,'slotformat':str,'creative':str,'slotid2':str})

i=0
for key in dic.keys():
  dic[key]=i
  i+=1
with open(folder+'positive'+'/'+'dic_'+'positive'+'.json', 'w') as f:
  json.dump(dic, f)

batch_size=int(72000/len(dic.keys()))*10000
print('batch size',batch_size)
category_columns=[column for column in datasets['train'].columns if not ('usertag' in column or 'click' in column or 'payprice' in column)]
binary_columns=[column for column in datasets['train'].columns if 'usertag' in column]
n_category_columns=len(category_columns)
n_binary_category_columns=dic[binary_columns[0]]

for dataset in ['validation','test']:
  data=datasets[dataset]
  print(dataset,len(data))
  if not dataset=='train':
    data.slotprice=np.ceil(np.minimum(data.slotprice,210)/slotprice_bin_width).astype(int)
  if not dataset=='test':
    click=np.asarray(data.click).astype(bool)
    data.payprice=np.ceil(np.minimum(data.payprice,300)/payprice_bin_width).astype(int)
    payprice=np.zeros([len(data),int(300/payprice_bin_width)+1]).astype(bool)
    payprice[range(len(payprice)),data.payprice]=True
    data=data.drop(columns=['click','payprice'])
    click.tofile(folder+'positive'+'/'+'_'.join([dataset,'original','click'])+'.bin')
    payprice.tofile(folder+'positive'+'/'+'_'.join([dataset,'original','payprice',str(payprice.shape[1])])+'.bin')
    print('click and payprice saved')
  data=np.asarray(data)
  for i in range(int(np.ceil(len(data)/float(batch_size)))):
    x=data[i*batch_size:min((i+1)*batch_size,len(data))]
    binary_x=np.zeros([len(x),n_binary_category_columns]).astype(bool)
    for j in range(len(x)):
      for k in range(n_category_columns):
        try:
          binary_x[j,dic[category_columns[k]+'_'+str(x[j,k])]]=True
        except:
          pass
    binary_x=np.hstack((binary_x,x[:,n_category_columns:])).astype(bool)
    binary_x.tofile(folder+'positive'+'/'+'_'.join([dataset,'original','x',str(i),str(len(x))])+'.bin')
    print(i,'x')

column "domain" has 476 unique instances
column "slotid2" has 327 unique instances
Total unique feature instances: 1660
batch size 430000
validation 303925
click and payprice saved
0 x
test 303375
0 x


In [0]:
data=pd.read_csv('data2/train1.csv')

In [0]:
np.count_nonzero(data.usertag_10048==0)

2288146

In [0]:
np.count_nonzero(data.usertag_10052==0)

2094156